## 모델 저장과 복원
- 방법 1) 모델 전체 저장(모델 구조 + 가중치 + 변수)
    * save_model()
    * 복원시 추가 작업 필요없이 모델 파일을 로딩해서 사용 가능
    
- 방법 2) 가중치만 저장
    * save_weights()
    * 복원 시 모델 구조(Architechure) 생성한 후 가중치 적용

- 방법 3) 모델 전체, 가중치 자동 저장
    * fit()에서 ModelCheckPoint Event에 대한 callback 등록
    * save_best_only = True : 모니터링 기준에 따라서 좋은 성능의 모델만 저장
    * save_weight_only = True : 가중치만 저장

- 파일 또는 폴더
    * 파일 확장자가 없으면 폴더로 저장
    * 파일 확장자
        - h5 / hdf5 : HDF5 포맷으로 모델 또는 가중치를 저장
        - ckpf : 체크파일 형태로 저장
        - pd : 모델 저장

In [1]:
from sklearn.datasets import load_iris
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import set_random_seed, plot_model
import numpy as np

## [1] 데이터 준비 및 로딩

In [2]:
iris = load_iris()

In [3]:
x_train = iris.data
y_train = iris.target

print(f'x_train.shape : {x_train.shape}, y_train.shape : {y_train.shape}')

x_train.shape : (150, 4), y_train.shape : (150,)


In [4]:
iris.target_names, iris.feature_names

(array(['setosa', 'versicolor', 'virginica'], dtype='<U10'),
 ['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'])

In [5]:
# Feature => 꽃잎의 길이, 너비
x_train = x_train[:, 2:]
x_train.shape

(150, 2)

## [2] 데이터 전처리

In [6]:
# Target => 문자를 수치로
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [7]:
# 라벨 인코딩
lencoder=LabelEncoder()
y_train=lencoder.fit(y_train).transform(y_train)

In [8]:
# 원 핫 인코딩
OHE=OneHotEncoder()
y_train=OHE.fit_transform(y_train.reshape(-1, 1))

In [9]:
# 스케일링
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
x_scaled_train=scaler.fit(x_train)

In [13]:
x_train.shape, y_train.shape, y_train[0]

((150, 2),
 (150, 3),
 <1x3 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>)

In [17]:
type(x_train), type(y_train)

(numpy.ndarray, scipy.sparse.csr.csr_matrix)

In [21]:
y_train=y_train.toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [19]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled_train, y_train,
                                                   stratify=y_train)

TypeError: Singleton array array(StandardScaler(), dtype=object) cannot be considered a valid collection.

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [ ]:
print(f'x_train.shape : {x_train.shape'})
print(f'x_test.shape : {x_test.shape'})
print(f'x_val.shape : {x_val.shape'})

## [3] 모델 구성 및 생성
- 입력 데이터 : 2개 Feature
- 출력 데이터 : 3개 품종
- 학습 방법 : 분류 - 다중분류

In [ ]:
irisModel=Sequential(name='irisModel')

In [ ]:
irisModel.add(Dense(9, activation='relu', input_shape=(2,)))
irisModel.add(Dense(3, activation='softmax'))

irisModel.summary()

In [ ]:
plot_model(irisModel)

In [ ]:
## 모델 생성
irisModel.compile(loss='categoricl_crossentropy',
                 optimizer='adam',
                 metrics='accuracy')

## [4] 모델 학습

In [ ]:
irisModel.fit(x_train, y_train, epochs=100)

## [5-1] 평가

In [ ]:
irisModel.evaluate(x_test, y_test)

In [ ]:
loosv, accv = irisModel.evaluate(x_test, y_test)

In [ ]:
lossv, accv

## [5-2] 모델 저장

In [ ]:
# 모델 저장
from tensorflow.keras.models import save_model

In [ ]:
MODEL_FILE = 'irisModel.h5'
WEIGHT_FILE = 'irisWeight'

In [ ]:
if accv >= 0.98:
    save_model(irisModel, MODEL_FILE)
    irisModel.save_weights(WEIGHT_FILE)

## [6] 모델 복원
- 모델 또는 가중치 복원
- load_model(모델 파일 또는 모델 폴더명)
- Sequential.load_weights(가중치 파일 또는 폴더명)

#### [6-1] 모델 복원

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
reModel=load_model(MODEL_FILE)

In [ ]:
reModel.evaluate(x_test, y_test)

#### [6-2] 가중치 복원
- 모델 구조 구성 및 생성
- 가중치 로딩

In [ ]:
# 모델 구조 구성
newModel=Sequential(name='NewIrisModel')
irisModel.add(Dense(9, activation='relu', input_shape=(2,)))
irisModel.add(Dense(3, activation='softmax'))

irisModel.summary()

In [ ]:
# 모델 생성
irisModel.compile(loss='categoricl_crossentropy',
                 optimizer='adam',
                 metrics='accuracy')

In [ ]:
# 가중치 로딩
newModel.load_weights(WEIGHT_FILE)

In [ ]:
newModel.evaluate(x_test, y_test)